In [1]:
!pip install --upgrade openai gradio pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


LLM SETUP

In [2]:
import openai

# Your API key
api_key = "sk-proj--"  # Replace with your OpenAI key

client = openai.OpenAI(api_key=api_key)

# Function to query ChatGPT
def query_chatgpt(prompt, model="gpt-4", temperature=0.7):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=1024
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

prompt = f"""generate a attack tree for bluetooth use mermaid format """
print(query_chatgpt(prompt))


Here is a basic example of an attack tree for Bluetooth using Mermaid format:

```mermaid
graph TD
    A[Bluetooth] --> B[Unauthorized Access]
    A --> C[Interception]
    A --> D[Denial of Service]
    B --> E[Brute Force Attack]
    B --> F[Guessing PIN]
    C --> G[Man in the Middle Attack]
    C --> H[Eavesdropping]
    D --> I[Jamming]
    D --> J[Resource Exhaustion]
    E --> K[Dictionary Attack]
    E --> L[Hybrid Attack]
    F --> M[Social Engineering]
    G --> N[Impersonation]
    G --> O[Replay Attack]
    H --> P[Passive Listening]
    I --> Q[Signal Interference]
    J --> R[Malicious Software]
    K --> S[Precomputed Hashes]
    L --> T[Combination of dictionary and brute force]
    M --> U[Phishing]
    N --> V[Creating Fake Device]
    O --> W[Capture and Resend Data]
    P --> X[Analysis of Traffic]
    Q --> Y[High Energy Consumption]
    R --> Z[Malware Injection]
    S --> AA[Using Rainbow Tables]
    T --> AB[Using Common Passwords and Dictionary Words]
    U -->

UI CODE with integrated AI

In [3]:
import openai
from openai import OpenAI
import gradio as gr
from pymongo import MongoClient
from datetime import datetime
import re
import csv
import os
import pandas as pd

# API & DB Config
OPENAI_API_KEY = "sk-proj-"#replace with chatgpt api
MONGODB_URI = "mongodb+srv:"# replace with mongodb atlas uri or use localhost

client_ai = OpenAI(api_key=OPENAI_API_KEY)
mongo_client = MongoClient(MONGODB_URI)
db = mongo_client["threat_db"]
collection = db["attack_trees"]

EXPORT_DIR = "csv_exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

def parse_mermaid_to_named_edges(mermaid_code):
    """
    Parses mermaid code and returns list of (parent_label, child_label)
    using node label mapping like A[Node Name]
    """
    node_labels = {}
    edges = []

    lines = mermaid_code.splitlines()

    # First pass: extract node labels like A[Label]
    for line in lines:
        node_match = re.findall(r'(\w+)\[(.+?)\]', line)
        for node_id, label in node_match:
            node_labels[node_id.strip()] = label.strip()

    # Second pass: extract edges like A --> B
    edge_pattern = re.compile(r'(\w+)\s*-->\s*(\w+)')
    for line in lines:
        match = edge_pattern.search(line)
        if match:
            parent_id = match.group(1).strip()
            child_id = match.group(2).strip()
            parent_label = node_labels.get(parent_id, parent_id)
            child_label = node_labels.get(child_id, child_id)
            edges.append((parent_label, child_label))

    return edges

def export_to_csv(prompt, edges):
    """
    Save edges to CSV with readable labels
    """
    filename = f"{prompt[:30].replace(' ', '_').replace('/', '_')}_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.csv"
    filepath = os.path.join(EXPORT_DIR, filename)

    with open(filepath, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Prompt", "Parent", "Child"])
        for parent, child in edges:
            writer.writerow([prompt, parent, child])

    return filepath

def read_csv_as_dataframe(csv_path):
    try:
        df = pd.read_csv(csv_path)
        return df
    except Exception:
        return pd.DataFrame(columns=["Prompt", "Parent", "Child"])

def generate_attack_tree(prompt):
    if not prompt.strip():
        return "❌ Please enter a valid prompt", pd.DataFrame()

    try:
        system_message = {
            "role": "system",
            "content": (
                "You are a cybersecurity expert. For any input threat scenario, respond with only the attack tree in valid Mermaid syntax using 'graph TD'. "
                "Do not include any explanation, just output:\n```mermaid\ngraph TD\n...\n```"
            )
        }

        response = client_ai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                system_message,
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=800
        )

        output = response.choices[0].message.content.strip()

        if output.startswith("```mermaid"):
            output = output.replace("```mermaid", "").replace("```", "").strip()

        # Save to MongoDB
        collection.update_one(
            {"prompt": prompt},
            {"$set": {"mermaid_code": output, "updated_at": datetime.utcnow()}},
            upsert=True
        )

        # Parse labeled edges & export
        edges = parse_mermaid_to_named_edges(output)
        csv_path = export_to_csv(prompt, edges)
        df = read_csv_as_dataframe(csv_path)

        return f"```mermaid\n{output}\n```", df

    except Exception as e:
        return f"❌ Error: {str(e)}", pd.DataFrame()

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("##  Threat Tree ")
    prompt_input = gr.Textbox(label="Enter Threat Prompt", lines=5, placeholder="e.g. Generate attack tree for CAN bus injection")
    mermaid_output = gr.Markdown(label="📌 Mermaid Diagram")
    csv_table = gr.Dataframe(headers=["Prompt", "Parent", "Child"], datatype=["str", "str", "str"], interactive=False)

    generate_button = gr.Button("🚀 Generate & Save")
    generate_button.click(fn=generate_attack_tree, inputs=prompt_input, outputs=[mermaid_output, csv_table])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://44d1257c69872e9da9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Fetching data from MONGODB

In [4]:
from pymongo import MongoClient
from pprint import pprint

# Replace with your actual MongoDB URI
MONGODB_URI = "mongodb+srv:"

# Connect to MongoDB
client = MongoClient(MONGODB_URI)
db = client["threat_db"]
collection = db["attack_trees"]

print("📦 Last 1 entries in MongoDB:")
for doc in collection.find().sort("created_at", -1).limit(5):
    pprint(doc)

📦 Last 1 entries in MongoDB:
{'_id': ObjectId('68260013204917402399b1c8'),
 'created_at': datetime.datetime(2025, 5, 15, 14, 54, 11, 526000),
 'mermaid_code': 'graph TD\n'
                 '    A[Successful Attack on CAN Bus] --> B[Message '
                 'Injection]\n'
                 '    A --> C[Denial of Service]\n'
                 '    A --> D[Spoofing]\n'
                 '    A --> E[Eavesdropping]\n'
                 '    A --> F[Fault Injection]\n'
                 '\n'
                 '    B --> G[Send Unauthorized Commands]\n'
                 '    B --> H[Alter Vehicle Behavior]\n'
                 '\n'
                 '    C --> I[Disable Vehicle Communication]\n'
                 '    C --> J[Prevent Emergency Signals]\n'
                 '\n'
                 '    D --> K[Trigger False Alarms]\n'
                 '    D --> L[Distrupt Operation]\n'
                 '\n'
                 '    E --> M[Gain Sensitive Information]\n'
                 '    E --> N[Unde